In [ ]:
import math

def max_deflection(spanl, roof = None, membertype='floor'):
    """
    NSCP 2015 Table 424.2.2. Maximum permissible calculated deflection.
    
    Parameters:
    spanl: Span length (mm)
    roof: Roof support condition (if applicable)
    membertype:
        'flatroofs' = Flat roof members
        'floor' = Floor members
        'roofdamaged' = Roof or floors that are LIKELY to be DAMAGED by large deflections
        'roofundamaged' = Roof or floors that are NOT LIKELY to be DAMAGED by large deflections
    """
    
    if membertype == 'flatroofs':
        return round(max(spanl / 180, (roof / 180) if roof else 0), 2)
    elif membertype == 'floor':
        return round(spanl / 360, 2)
    elif membertype == 'roofdamaged':
        return round(spanl / 480, 2)
    elif membertype == 'roofundamaged':
        return round(spanl / 240, 2)
    else:
        raise ValueError("Invalid member type")

def effective_moment_of_inertia(mcr, ma, ig, icr):
    """
    NSCP 2015 Eq. 424.2.3.5a. Effective moment of inertia
    
    Parameters:
    mcr: Cracking moment (N-mm)
    ma: Applied moment (N-mm)
    ig: Gross moment of inertia (mm^4)
    icr: Cracked section moment of inertia (mm^4)
    """
    
    ratio = (mcr / ma) ** 3
    results = (ratio * ig) + ((1 - ratio) * icr)
    return round(results, 2)

def time_dependent_deflection(idef, xi, rho):
    """
    NSCP 2015 Table 424.2.4.1.3. Time-dependent deflection for non-prestressed members.
    
    Parameters:
    idef: Immediate deflection due to sustained load (mm)
    xi: Time-dependent factor based on sustained load duration
    rho: Reinforcement ratio
    """
    
    lambda_delta = xi / (1 + 50 * rho)
    results = idef * lambda_delta
    return round(results, 2)

def cracking_moment(fcp, LAMBDA, fr, ig, yt):
    """
    NSCP 2015 Eq. 424.2.3.5b. Cracking moment  
    
    Parameters:
    fcp: Concrete compressive strength (MPa)
    LAMBDA: Concrete density modification factor
    fr: Modulus of rupture (MPa)
    ig: Gross moment of inertia (mm^4)
    yt: Distance from neutral axis to extreme fiber (mm)
    """
    
    Mcr = (fr * ig) / yt
    return round(Mcr, 2)

# Input
if __name__ == "__main__":
    spanl = 6000  # mm 
    idef = 5  # mm
    xi = 1.4  # for 12 months sustained load
    rho = 0.002  # example reinforcement ratio
    ma = 5e9  # Applied moment (N-mm)
    ig = 2e9  # Gross moment of inertia (mm^4)
    icr = 0.5e9  # Cracked section moment of inertia (mm^4)
    yt = 40  # mm (Distance from neutral axis to extreme fiber)
    
    # Concrete parameters
    fcp = 17  # MPa (Concrete compressive strength)
    LAMBDA = 1.00  # Normal-weight concrete (NSCP 2015 Table 419.2.4.2)
    fr = 0.62 * LAMBDA * math.sqrt(fcp)  # Modulus of rupture

    # Compute cracking moment
    mcr = cracking_moment(fcp, LAMBDA, fr, ig, yt)
    
print("Max Deflection (Floors):", max_deflection(spanl, membertype = 'floor'), "mm")
print("Effective Moment of Inertia:", effective_moment_of_inertia(mcr, ma, ig, icr), "mm⁴")
print("Time-Dependent Deflection:", time_dependent_deflection(idef, xi, rho), "mm")
print("Cracking Moment:", mcr, "N-mm")